# Hackaton

In [41]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import joblib

if mlflow.active_run() is not None:
    mlflow.end_run()
# Configurar el servidor de MLflow
mlflow.set_tracking_uri("http://localhost:5001/")  # Reemplaza con tu URI
mlflow.set_experiment("Mood-Based Playlist Experiment")

# Configurar autenticación de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="TU_CLIENT_ID",       # Reemplaza con tu Client ID
    client_secret="TU_CLIENT_SECRET",  # Reemplaza con tu Client Secret
    redirect_uri="http://localhost:9443/callback",  # URL de redirección configurada
    scope="playlist-modify-public"
))

# Paso 1: Cargar el dataset
file_path = 'data/dataset.csv'  # Ruta al archivo cargado
spotify_data = pd.read_csv(file_path)

# Selección de columnas clave
selected_columns = ['track_name', 'artists', 'popularity', 'explicit', 'danceability', 'energy', 'valence', 'track_genre']
spotify_data = spotify_data[selected_columns]

# Preprocesamiento
spotify_data = spotify_data.dropna()
label_encoder = LabelEncoder()
spotify_data['track_genre'] = label_encoder.fit_transform(spotify_data['track_genre'])
scaler = MinMaxScaler()
numerical_features = ['popularity', 'danceability', 'energy', 'valence']
spotify_data[numerical_features] = scaler.fit_transform(spotify_data[numerical_features])

# Crear una columna de estado de ánimo como objetivo (target)
def classify_mood(valence, energy, danceability):
    if valence > 0.75 and energy > 0.7 and danceability > 0.7:
        return 'Euforia'
    elif valence > 0.7 and energy > 0.6 and danceability > 0.6:
        return 'Alegría'
    elif valence < 0.3 and energy < 0.4:
        return 'Melancolía'
    elif valence < 0.4 and energy < 0.5 and danceability < 0.5:
        return 'Tristeza'
    elif 0.4 <= valence <= 0.6 and 0.5 <= energy <= 0.7 and danceability > 0.6:
        return 'Relajación'
    elif energy > 0.8 and danceability > 0.8:
        return 'Fiesta'
    else:
        return 'Otro'

spotify_data['mood'] = spotify_data.apply(
    lambda row: classify_mood(row['valence'], row['energy'], row['danceability']),
    axis=1
)

# Dividir en conjuntos de entrenamiento y prueba
X = spotify_data[numerical_features + ['track_genre']]
y = spotify_data['mood']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenar y registrar en MLflow
with mlflow.start_run() as run:
    mlflow.log_param("model_type", "Random Forest")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)

    mlflow.set_tags({
        "project_name": "Mood-Based Playlist Generator",
        "author": "Your Name",
        "model_description": "Random Forest trained on mood classification",
        "dataset_source": "Spotify tracks data"
    })

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Registrar el modelo en MLflow
    model_name = "Mood-Based Playlist Model"  # Nombre representativo para el modelo
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Registrar el modelo en el Model Registry

    try:
        # Registrar el modelo en el Model Registry
        client = MlflowClient()
        result = client.create_registered_model(model_name)
        client.create_model_version(
            name=model_name,
            source=f"runs:/{run.info.run_id}/random_forest_model",
            run_id=run.info.run_id
        )
        print(f"Modelo '{model_name}' registrado exitosamente.")
    except mlflow.exceptions.RestException as e:
        if "RESOURCE_ALREADY_EXISTS" in str(e):
            client.create_model_version(
                name=model_name,  # Modelo existente
                source=f"runs:/{run.info.run_id}/random_forest_model",
                run_id=run.info.run_id
            )
            print(f"Nueva versión creada para el modelo registrado '{model_name}'.")

    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    for mood, metrics in report.items():
        if isinstance(metrics, dict):
            for metric_name, metric_value in metrics.items():
                mlflow.log_metric(f"{mood}_{metric_name}", metric_value)

    scaler_path = "backend/models/playlist-model/scaler.joblib"
    label_encoder_path = "backend/models/playlist-model/label_encoder.joblib"
    
    # Serializar y guardar
    joblib.dump(scaler, scaler_path)
    joblib.dump(label_encoder, label_encoder_path)
    print(f"Modelo '{model_name}' registrado exitosamente en el Model Registry de MLflow.")


2024/12/03 15:03:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/12/03 15:03:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Mood-Based Playlist Model, version 2


Nueva versión creada para el modelo registrado 'Mood-Based Playlist Model'.
Modelo 'Mood-Based Playlist Model' registrado exitosamente en el Model Registry de MLflow.
🏃 View run exultant-wren-266 at: http://localhost:5001/#/experiments/5/runs/99cf0c22a76b47f698221266c5dd3e99
🧪 View experiment at: http://localhost:5001/#/experiments/5


In [ ]:
# Interacción con el usuario
print("\nOpciones de sentimientos disponibles:", spotify_data['mood'].unique())
user_mood = input("Introduce el sentimiento para tu playlist: ")

In [ ]:

# Generar playlist basada en la predicción del modelo
def generate_playlist_by_mood(spotify_data, mood):
    print(f"\nGenerando playlist para el sentimiento: {mood}")
    playlist = spotify_data[spotify_data['mood'] == mood].sort_values(by='popularity', ascending=False)
    return playlist[['track_name', 'artists', 'popularity', 'track_genre', 'explicit']]

print("\nGenerando playlist para el sentimiento: ", user_mood)
playlist = generate_playlist_by_mood(spotify_data, user_mood)

# Mostrar las primeras canciones de la playlist generada
if not playlist.empty:
    print(f"\nPlaylist generada basada en {user_mood}:")
    print(playlist.head(10))
else:
    print(f"\nNo se encontraron canciones para el sentimiento: {user_mood}")


